In [ ]:
# GETTING READY FOR NICKNAME-TAGGING, IN CASE THAT THERE ARE NOT ENOUGH NAMES MENTIONED

In [1]:
# Numpy 다운그레이드; '세션 재시작'해야 할 수 있음 ---> (세션 재시작 시,) "패키지 설치" 단계는 다시 실행하면 안됨!
# 주의!!! '세션 재시작'이 아니라, '런타임 연결 해체 및 삭제' 후 다시 연결한 것이라면 "패키지 설치" 단계를 반드시 다시 실시해야 함!
!pip install numpy==1.26.4

print("\n🔴 [필수] 위 설치가 완료되면 상단 메뉴 '런타임' > '세션 다시 시작'을 클릭하세요!")
print("🔴 그 후, 이 셀은 건너뛰고 다음 셀(폴더 세팅)부터 실행하세요.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-he


🔴 [필수] 위 설치가 완료되면 상단 메뉴 '런타임' > '세션 다시 시작'을 클릭하세요!
🔴 그 후, 이 셀은 건너뛰고 다음 셀(폴더 세팅)부터 실행하세요.


In [1]:
# === 폴더 세팅 ===
import os, subprocess, json, math, re, pandas as pd
BASE = "/content"
AUDIO_DIR = f"{BASE}/audio"
WORK = f"{BASE}/work"
OUT = f"{WORK}/outputs"
CACHE = f"{WORK}/cache"
for d in [AUDIO_DIR, OUT, CACHE]:
    os.makedirs(d, exist_ok=True)



In [2]:
# === 오디오 파일 설정 (드라이브 연동 or 직접 업로드) ===
import os, shutil
from google.colab import drive
from google.colab import files

# 1. 기본 설정
target_filename = "1107 애슐리_소음"  # 확장자 제외한 파일명 (팀원들과 합의된 이름)
extensions = [".m4a", ".wav", ".mp3", ".aac", ".flac", ".ogg"]

# 2. 드라이브 마운트 시도
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 3. 파일 찾기 (경로를 좀 더 유연하게 설정하거나, 없으면 업로드 요청)
# 팀원마다 경로가 다를 수 있으므로, 드라이브 내 특정 경로를 찾지 못하면 업로드하게 유도합니다.
drive_dir = "/content/drive/MyDrive/final project/audio files" # 팀원 공통 경로
found_path = None

# 드라이브에서 파일 탐색
if os.path.exists(drive_dir):
    for ext in extensions:
        check_path = os.path.join(drive_dir, target_filename + ext)
        if os.path.exists(check_path):
            found_path = check_path
            break

# 4. 결과 처리
if found_path:
    print(f"✅ 구글 드라이브에서 파일을 찾았습니다: {found_path}")
    dst_path = os.path.join(AUDIO_DIR, os.path.basename(found_path))
    shutil.copy2(found_path, dst_path)
    AUDIO_IN = dst_path
else:
    print(f"❌ 드라이브 경로에 '{target_filename}' 파일이 없습니다.")
    print("👇 아래 버튼을 눌러 오디오 파일을 직접 업로드해주세요.")
    uploaded = files.upload()
    if uploaded:
        fname = list(uploaded.keys())[0]
        AUDIO_IN = os.path.join(BASE, fname) # 업로드된 파일 경로
        # audio 폴더로 이동
        shutil.move(AUDIO_IN, os.path.join(AUDIO_DIR, fname))
        AUDIO_IN = os.path.join(AUDIO_DIR, fname)
    else:
        raise FileNotFoundError("파일이 업로드되지 않았습니다.")

print(f"🎯 최종 분석 대상 파일: {AUDIO_IN}")

Mounted at /content/drive
✅ 구글 드라이브에서 파일을 찾았습니다: /content/drive/MyDrive/final project/audio files/1107 애슐리_소음.m4a
🎯 최종 분석 대상 파일: /content/audio/1107 애슐리_소음.m4a


In [3]:
# --- 패키지 설치 (수정된 순서) ---
# '세션 재시작' 후 다시 시작 금지!!!! but '런타임 연결해제' 후 런타임 재연결했으면 그때는 꼭 실행해야 함!

print("--- 1. Colab 기본 패키지 호환성 설정 (Pandas) ---")
# google-colab이 요구하는 pandas==2.2.2 버전을 먼저 설치합니다.
# (이전 에러 메시지에서 충돌의 주 원인이었습니다.)
!pip install pandas==2.2.2

print("--- 2. Diarization 모듈 설치 (NVIDIA NeMo) ---")
# 의존성이 가장 무겁고 PyTorch 버전에 민감한 NeMo를 먼저 설치합니다.
# NeMo가 필요한 torch, numpy 버전을 자동으로 설치하도록 합니다.
!pip -q install "nemo_toolkit[asr]"

print("--- 3. STT 모듈 설치 (Whisper) ---")
# NeMo가 설치한 PyTorch 환경 위에서 STT 라이브러리를 설치합니다.
# --upgrade 옵션은 NeMo의 PyTorch를 변경시킬 수 있으므로 제외하거나 주의합니다.
!pip install -q openai-whisper faster-whisper torchaudio librosa soundfile

print("--- 설치 완료 ---")

--- 1. Colab 기본 패키지 호환성 설정 (Pandas) ---
--- 2. Diarization 모듈 설치 (NVIDIA NeMo) ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.7/95.7 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91

In [4]:
# 전처리 — 16kHz 모노 WAV 생성

# ffmpeg로 16kHz mono wav 생성
WAV_16K = f"{CACHE}/meeting_16k.wav"
!ffmpeg -y -i "$AUDIO_IN" -ac 1 -ar 16000 "$WAV_16K" >/dev/null 2>&1
print("WAV_16K =", WAV_16K)


WAV_16K = /content/work/cache/meeting_16k.wav


In [5]:
# Diarization
# 5-1) 일반 inference용 YAML 다운로드
!wget -qO {WORK}/diar_infer_general.yaml \
  https://raw.githubusercontent.com/NVIDIA/NeMo/stable/examples/speaker_tasks/diarization/conf/inference/diar_infer_general.yaml

# 5-2) NeMo manifest 생성(JSON lines)
import json
manifest = f"{WORK}/manifest.json"
meta = {
    "audio_filepath": WAV_16K,
    "offset": 0, "duration": None,
    "label": "infer", "text": "-",
    "num_speakers": None,      # 모르면 None, 알면 정수(예: 5)
    "rttm_filepath": None, "uem_filepath": None
}
with open(manifest, "w") as f:
    f.write(json.dumps(meta) + "\n")

# 5-3) ClusteringDiarizer 실행
from omegaconf import OmegaConf
from nemo.collections.asr.models import ClusteringDiarizer

cfg = OmegaConf.load(f"{WORK}/diar_infer_general.yaml")
cfg.diarizer.manifest_filepath = manifest
cfg.diarizer.out_dir = OUT
# 필요 시 상한만 지정(선택): cfg.diarizer.clustering.parameters.max_num_speakers = 10

sd = ClusteringDiarizer(cfg=cfg)
sd.diarize()
print("NeMo diarization done. OUT dir:", OUT)


[NeMo W 2025-11-14 06:36:20 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.
[NeMo W 2025-11-14 06:36:23 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-11-14 06:36:23 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-11-14 06:36:23 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-11-14 06:36:23 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


[NeMo I 2025-11-14 06:36:25 nemo_logging:393] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2025-11-14 06:36:25 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_2.5.3/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2025-11-14 06:36:25 nemo_logging:393] Instantiating model from pre-trained checkpoint


[NeMo W 2025-11-14 06:36:25 nemo_logging:405] Please use the EncDecSpeakerLabelModel instead of this model. EncDecClassificationModel model is kept for backward compatibility with older models.
[NeMo W 2025-11-14 06:36:25 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_

[NeMo I 2025-11-14 06:36:25 nemo_logging:393] PADDING: 16
[NeMo I 2025-11-14 06:36:26 nemo_logging:393] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.5.3/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2025-11-14 06:36:26 nemo_logging:393] Loading pretrained titanet_large model from NGC
[NeMo I 2025-11-14 06:36:26 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_2.5.3/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2025-11-14 06:36:32 nemo_logging:393] Instantiating model from pre-trained checkpoint


[NeMo W 2025-11-14 06:36:33 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2025-11-14 06:36:33 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data

...

[NeMo I 2025-11-14 06:37:14 nemo_logging:393] Saved embedding files to /content/work/outputs/speaker_outputs/embeddings


clustering: 100%|██████████| 1/1 [00:02<00:00,  2.81s/it]

[NeMo I 2025-11-14 06:37:17 nemo_logging:393] Outputs are saved in /content/work/outputs directory



[NeMo W 2025-11-14 06:37:17 nemo_logging:405] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


NeMo diarization done. OUT dir: /content/work/outputs


In [6]:
# RTTM 파일 찾기 & 파싱

import glob, os, json

# OUT는 앞에서 정의한 /content/work/outputs
rttms = sorted(glob.glob(f"{OUT}/**/*.rttm", recursive=True), key=os.path.getmtime)
assert rttms, "RTTM not found under OUT. Check diarization step."
rttm_path = rttms[-1]
print("Using RTTM:", rttm_path)

def load_rttm(rttm_path):
    segs = []
    with open(rttm_path,"r") as f:
        for line in f:
            p = line.strip().split()
            if not p or p[0]!="SPEAKER" or len(p)<9:
                continue
            start = float(p[3]); dur = float(p[4]); spk = p[7]
            segs.append({"speaker": spk, "start": start, "end": start+dur})
    return sorted(segs, key=lambda x: x["start"])

segments = load_rttm(rttm_path)
print("segments:", len(segments), "| sample:", segments[:3])


Using RTTM: /content/work/outputs/pred_rttms/meeting_16k.rttm
segments: 1100 | sample: [{'speaker': 'speaker_2', 'start': 0.0, 'end': 0.625}, {'speaker': 'speaker_0', 'start': 0.625, 'end': 3.375}, {'speaker': 'speaker_1', 'start': 3.375, 'end': 4.85}]


In [7]:
# STT 단어 파일(없으면 지금 생성): 이미 Whisper를 돌려 stt_words.json이 있다면 로드만 하세요. 없다면 이 셀로 곧장 만듭니다.

import os, json

stt_words_path = f"{OUT}/stt_words.json"
if not os.path.exists(stt_words_path):
    import whisper
    whisp = whisper.load_model("medium")  # VRAM 여유면 "large-v3"
    stt = whisp.transcribe(WAV_16K, language="ko", word_timestamps=True)
    words = []
    for seg in stt.get("segments", []):
        if "words" in seg:
            for w in seg["words"]:
                words.append({"text": w["word"], "start": float(w["start"]), "end": float(w["end"])})
        else:
            words.append({"text": seg["text"], "start": float(seg["start"]), "end": float(seg["end"])})
    with open(stt_words_path,"w",encoding="utf-8") as f:
        json.dump(words, f, ensure_ascii=False, indent=2)
    print("Created:", stt_words_path, "count:", len(words))
else:
    print("Found:", stt_words_path)


100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 243MiB/s]


Created: /content/work/outputs/stt_words.json count: 1953


In [8]:
# 병합 (단어 --- 화자, Max-overlap)

import bisect, json

with open(stt_words_path,"r",encoding="utf-8") as f:
    words = json.load(f)

starts = [s["start"] for s in segments]
def find_speaker(ws, we):
    i = max(0, bisect.bisect_right(starts, ws)-1)
    best, best_ov = None, 0.0
    for j in range(i, min(i+6, len(segments))):
        s = segments[j]
        ov = max(0.0, min(we, s["end"]) - max(ws, s["start"]))
        if ov > best_ov:
            best_ov, best = ov, s["speaker"]
        if s["start"] > we: break
    return best or "UNKNOWN"

for w in words:
    w["speaker"] = find_speaker(w["start"], w["end"])

utter = []
if words:
    cur = {"speaker": words[0]["speaker"], "start": words[0]["start"], "end": words[0]["end"], "text": words[0]["text"].strip()}
    for w in words[1:]:
        contiguous = (w["speaker"] == cur["speaker"]) and (w["start"] - cur["end"] <= 0.8)
        if contiguous:
            cur["text"] += ("" if cur["text"].endswith((" ", "")) else " ") + w["text"].strip()
            cur["end"]  = w["end"]
        else:
            utter.append(cur);
            cur = {"speaker": w["speaker"], "start": w["start"], "end": w["end"], "text": w["text"].strip()}
    utter.append(cur)

with open(f"{OUT}/final_transcript.json","w",encoding="utf-8") as f:
    json.dump(utter, f, ensure_ascii=False, indent=2)

print("final_transcript.json saved:", len(utter))
print("peek:", utter[:2])


final_transcript.json saved: 613
peek: [{'speaker': 'speaker_1', 'start': 7.300000000000001, 'end': 9.98, 'text': '맛있게드세요잘'}, {'speaker': 'speaker_0', 'start': 9.98, 'end': 14.6, 'text': '드세요'}]


In [ ]:
# (선택사항)
# 빠른 점검 & SRT 출력

# 스피커별 총 발화 시간/턴 수
from collections import defaultdict
stats = defaultdict(lambda: {"dur":0.0,"utt":0})
for u in utter:
    stats[u["speaker"]]["dur"] += (u["end"]-u["start"])
    stats[u["speaker"]]["utt"] += 1
print({k: (round(v["dur"],1), v["utt"]) for k,v in stats.items()})

# SRT 내보내기
def srt_ts(sec):
    h = int(sec//3600); sec -= h*3600
    m = int(sec//60); sec -= m*60
    s = int(sec); ms = int(round((sec - s)*1000))
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

srt_lines = []
for i,u in enumerate(utter,1):
    head = f"{srt_ts(u['start'])} --> {srt_ts(u['end'])}"
    text = f"[{u['speaker']}] {u['text'].strip()}"
    srt_lines += [str(i), head, text, ""]
srt_path = f"{OUT}/meeting.srt"
with open(srt_path,"w",encoding="utf-8") as f:
    f.write("\n".join(srt_lines))
print("SRT saved:", srt_path)


In [ ]:
# === 경로 복구 & 점검 ===
import os, json
BASE  = "/content"
WORK  = f"{BASE}/work"
OUT   = f"{WORK}/outputs"
CACHE = f"{WORK}/cache"
for d in (WORK, OUT, CACHE): os.makedirs(d, exist_ok=True)

final_json_path = f"{OUT}/final_transcript.json"
assert os.path.exists(final_json_path), f"필요 파일 없음: {final_json_path}"
print("OK:", final_json_path, "exists")


In [10]:
# =========================================
# Nickname-Tagging: 규칙 엔진 + 대표 발화 추출
# =========================================
import os, re, json, math, unicodedata
import numpy as np
import pandas as pd
from collections import defaultdict
from google.colab import files

OUT_DIR = OUT if 'OUT' in globals() else '/content/work/outputs'
os.makedirs(OUT_DIR, exist_ok=True)

# ----- 0) 유틸 -----
_PUNCT_RE = re.compile(r'[^\w\sㄱ-힣?!.~…]+')
def _clean(txt:str)->str:
    txt = unicodedata.normalize("NFKC", str(txt)).strip()
    return _PUNCT_RE.sub('', txt)

def _sec(x):
    try:
        return float(x)
    except:
        return np.nan

# ----- 1) 파서: 두 종류 포맷 자동 인식 -----
PAT_BRACKET = re.compile(
    r'^\[(?P<label>[^\]]+)\]:\s*\((?P<start>[\d.]+)s\s*-\s*(?P<end>[\d.]+)s\)\s*(?P<text>.*)$'
)
PAT_KR = re.compile(
    r'^(?:화자|스피커)\s*(?P<label>[A-Za-z0-9_가-힣]+)\s*:\s*(?P<text>.*)$'
)

def load_diarized_to_df(path:str)->pd.DataFrame:
    rows = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = [l.rstrip('\n') for l in f if l.strip()]
    mode = None
    # 포맷 감지
    for l in lines[:10]:
        if PAT_BRACKET.match(l):
            mode = 'bracket'; break
        if PAT_KR.match(l):
            mode = 'kr'; break
    if mode is None:
        raise ValueError("지원하지 않는 포맷입니다. 파일 앞부분을 확인해 주세요.")

    idx = 0
    for l in lines:
        if mode=='bracket':
            m = PAT_BRACKET.match(l)
            if not m: continue
            spk = m.group('label').strip()
            st  = _sec(m.group('start'))
            en  = _sec(m.group('end'))
            txt = _clean(m.group('text'))
            rows.append({'order':idx, 'speaker':spk, 'start':st, 'end':en, 'text':txt})
            idx += 1
        else:
            m = PAT_KR.match(l)
            if not m: continue
            spk = m.group('label').strip()
            txt = _clean(m.group('text'))
            rows.append({'order':idx, 'speaker':f'SPEAKER_{spk}', 'start':np.nan, 'end':np.nan, 'text':txt})
            idx += 1

    df = pd.DataFrame(rows)
    if df.empty:
        raise ValueError("파싱된 발화가 없습니다.")
    # 길이/단어수
    df['char_len'] = df['text'].str.len()
    df['word_len'] = df['text'].str.split().apply(len)
    # 구두점 제거한 짧은 토큰(백채널 판정에 사용)
    df['text_core'] = df['text'].str.replace(r'[^\w\sㄱ-힣]+','',regex=True).str.strip()

    # duration
    if 'start' in df and 'end' in df:
        df['dur'] = df['end'] - df['start']
    else:
        df['dur'] = np.nan
    return df.sort_values(['order']).reset_index(drop=True)

# ----- 2) 규칙 엔진을 위한 특징량 -----
BACKCHANNEL_LIST = [
    "네","넵","예","응","음","아","어","오","우와",
    "맞아","맞아요","맞습니다","그렇죠","그쵸","그럼요","좋아요",
    "허허","하하","ㅎㅎ","ㅋㅋ","아하","오키","오케이","옙","예예","네네"
]
def is_backchannel(s:str)->bool:
    s = s.strip()
    if not s: return False
    # 아주 짧고 감탄/동의류 위주
    if len(s) <= 8:
        for bc in BACKCHANNEL_LIST:
            if s == bc or s.startswith(bc):
                return True
    # 의성어/추임새만으로 구성된 경우
    if re.fullmatch(r'[ㅋㅎ아어음네예응오우와\s]+', s):
        return True
    return False

REQ_PAT = re.compile(r'(해줄\s*수|해주세요|부탁|가능할까요|가능\s*한가요|\?$)')
NEG_PAT = re.compile(r'(아니|못하|없[다어요]?|문제|에러|힘들|어렵|곤란|안\s*되)')
AGENDA_PAT = re.compile(r'(일단|먼저|다음|목표|계획|정리|요약|파이프라인|의제|안건)')
GUIDE_PAT  = re.compile(r'(자[, ]|그럼|진행|하시죠|합시다|해보죠|넘어가|말씀해)')

def summarize_rules(df:pd.DataFrame)->pd.DataFrame:
    # 타임라인 기반 보조 피처
    df = df.copy()
    df['is_back'] = df['text_core'].apply(is_backchannel)
    df['is_req']  = df['text'].str.contains(REQ_PAT)
    df['is_neg']  = df['text'].str.contains(NEG_PAT)
    df['is_agenda'] = df['text'].str.contains(AGENDA_PAT)
    df['is_guide']  = df['text'].str.contains(GUIDE_PAT)

    # 응답 지연(직전 타인 발화 → 이번 내 발화)
    df['resp_latency'] = np.nan
    last_end_by_any = None
    last_spk = None
    for i,row in df.iterrows():
        if not np.isnan(row.get('start', np.nan)) and last_end_by_any is not None and row['speaker']!=last_spk:
            df.loc[i,'resp_latency'] = row['start'] - last_end_by_any
        if not np.isnan(row.get('end', np.nan)):
            last_end_by_any = row['end']
        last_spk = row['speaker']

    # 화자별 집계
    g = df.groupby('speaker', as_index=False)
    tot_dur_all = df['dur'].dropna().sum()
    def _safe_share(x):
        return float(x)/tot_dur_all if (not np.isnan(x) and tot_dur_all>0) else np.nan

    agg = g.agg(
        num_utts=('text','count'),
        total_chars=('char_len','sum'),
        avg_chars=('char_len','mean'),
        total_words=('word_len','sum'),
        total_dur=('dur','sum'),
        back_cnt=('is_back','sum'),
        req_cnt=('is_req','sum'),
        neg_cnt=('is_neg','sum'),
        agenda_cnt=('is_agenda','sum'),
        guide_cnt=('is_guide','sum'),
        first_idx=('order','min'),
        last_idx=('order','max'),
        first_start=('start','min'),
        last_end=('end','max'),
        mean_latency=('resp_latency','mean')
    )
    # 비율
    agg['back_ratio'] = agg['back_cnt'] / agg['num_utts'].clip(lower=1)
    agg['req_ratio']  = agg['req_cnt']  / agg['num_utts'].clip(lower=1)
    agg['neg_ratio']  = agg['neg_cnt']  / agg['num_utts'].clip(lower=1)
    agg['talk_share'] = agg['total_dur'].apply(_safe_share)

    # 최대 무발화 간격(자신의 연속 턴 사이 gap)
    max_gaps = []
    for spk, d in g:
        d2 = d.sort_values('order')
        gaps = []
        prev_end = None
        for _,r in d2.iterrows():
            if prev_end is not None and not (np.isnan(prev_end) or np.isnan(r.get('start',np.nan))):
                gaps.append(r['start']-prev_end)
            prev_end = r.get('end', np.nan)
        max_gaps.append({'speaker':spk,'max_gap': max(gaps) if gaps else np.nan})
    agg = agg.merge(pd.DataFrame(max_gaps), on='speaker', how='left')

    # 규칙 라벨링
    roles = []
    for _,r in agg.iterrows():
        role_tags = []
        # 1) 진행자 후보
        if (r.get('guide_cnt',0)>=4) or \
           ((not np.isnan(r.get('talk_share',np.nan)) and r['talk_share']>=0.30) and r.get('agenda_cnt',0)>=3):
            role_tags.append('진행자')

        # 2) 백채널러
        if r['back_ratio']>=0.6 and r['avg_chars']<=12 and r['num_utts']>=3:
            role_tags.append('백채널러')

        # 3) 요청자(요청/질문 빈도 높음)
        if r['req_ratio']>=0.30 and r['num_utts']>=2:
            role_tags.append('요청자')

        # 4) 문제제기/우려 제시
        if r['neg_ratio']>=0.25 and r['avg_chars']>=20:
            role_tags.append('우려·문제제기')

        # 5) 잠깐참여
        if (r['num_utts']<=3 and (r['total_chars']<=40 or (not np.isnan(r['talk_share']) and r['talk_share']<0.05))):
            role_tags.append('잠깐참여')

        # 6) 기본 태그
        if not role_tags:
            if not np.isnan(r.get('talk_share',np.nan)) and r['talk_share']>=0.20:
                role_tags.append('핵심참여')
            else:
                role_tags.append('일반참여')

        roles.append({'speaker':r['speaker'], 'role_tags':role_tags})
    role_df = pd.DataFrame(roles)
    return agg.merge(role_df, on='speaker', how='left').sort_values(
        ['talk_share','num_utts','total_chars'], ascending=[False,False,False]
    ).reset_index(drop=True)

# ----- 3) 대표 발화 추출 -----
def pick_representatives(df:pd.DataFrame, per_speaker:int=2)->dict:
    reps = {}
    # 시간 축 percentile 계산(시간 없으면 order를 대용)
    has_time = df['start'].notna().any() and df['end'].notna().any()
    timeline_key = 'start' if has_time else 'order'
    # 백채널/매우 짧은 발화 제외
    cand = df[~df['text_core'].apply(is_backchannel)].copy()
    if cand.empty: cand = df.copy()
    # 스코어: 길이 우선 + (초반 20~40%/중반 40~70%)에서 하나씩
    if has_time:
        total_span = (np.nanmin(df['start']), np.nanmax(df['end']))
        span = total_span[1] - total_span[0] if not np.isnan(total_span[0]) and not np.isnan(total_span[1]) else None
        def pct(s):
            if span and not np.isnan(s):
                return (s - total_span[0]) / max(span, 1e-6)
            return np.nan
        cand['pos_pct'] = cand['start'].apply(pct)
    else:
        cand['pos_pct'] = cand['order'] / max(len(df), 1)

    reps = {}
    for spk, d in cand.groupby('speaker'):
        d2 = d.sort_values(['char_len'], ascending=False)
        early = d2.loc[(d2['pos_pct']>=0.20) & (d2['pos_pct']<=0.45)]
        mid   = d2.loc[(d2['pos_pct']>0.45) & (d2['pos_pct']<=0.75)]
        take = []
        if not early.empty:
            take.append(early.iloc[0])
        if not mid.empty and len(take)<per_speaker:
            take.append(mid.iloc[0])
        # 부족하면 그냥 긴 순으로 채우기
        if len(take)<per_speaker:
            for _,r in d2.iterrows():
                if any(r.name==t.name for t in take):
                    continue
                take.append(r)
                if len(take)>=per_speaker: break
        reps[spk] = [
            {
              'start': float(r['start']) if not np.isnan(r['start']) else None,
              'end':   float(r['end']) if not np.isnan(r['end']) else None,
              'text':  r['text']
            } for r in take
        ]
    return reps

# ----- 4) 실행: 파일 선택 → 파싱 → 요약/대표발화 → 저장/다운로드 -----
# 필요에 따라 한 개만 실행하고 싶으면 paths 리스트에서 하나만 남기세요.
paths = [
  "/content/drive/MyDrive/final project/diarized/1106_오후_회의_assemblyai1 (recommended result).txt",
  "/content/drive/MyDrive/final project/diarized/1106_오후_회의_senko (recommended result).txt",
]

all_outputs = []
for in_path in paths:
    print(f"\n### Processing: {in_path}")
    df = load_diarized_to_df(in_path)
    prof = summarize_rules(df)
    reps = pick_representatives(df, per_speaker=2)

    base = os.path.splitext(os.path.basename(in_path))[0]
    df_path      = os.path.join(OUT_DIR, f"{base}__utterances.parquet")
    prof_csv     = os.path.join(OUT_DIR, f"{base}__speaker_profiles.csv")
    prof_json    = os.path.join(OUT_DIR, f"{base}__speaker_profiles.json")
    reps_json    = os.path.join(OUT_DIR, f"{base}__representatives.json")

    df.to_parquet(df_path, index=False)
    prof.to_csv(prof_csv, index=False, encoding='utf-8')
    with open(prof_json,'w',encoding='utf-8') as f:
        json.dump(json.loads(prof.to_json(orient='records', force_ascii=False)), f, ensure_ascii=False, indent=2)
    with open(reps_json,'w',encoding='utf-8') as f:
        json.dump(reps, f, ensure_ascii=False, indent=2)

    # 즉시 다운로드(Colab → 로컬)
    for p in [prof_csv, prof_json, reps_json]:
        try:
            files.download(p)
        except Exception as e:
            print(f"(download skip) {p}: {e}")

    all_outputs.append({
        'input': in_path,
        'utterances_parquet': df_path,
        'speaker_profiles_csv': prof_csv,
        'speaker_profiles_json': prof_json,
        'representatives_json': reps_json
    })

print("\nDone. Summary:")
print(pd.DataFrame(all_outputs))



### Processing: /content/drive/MyDrive/final project/diarized/1106_오후_회의_assemblyai1 (recommended result).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Done. Summary:
                                               input  \
0  /content/drive/MyDrive/final project/diarized/...   
1  /content/drive/MyDrive/final project/diarized/...   

                                  utterances_parquet  \
0  /content/work/outputs/1106_오후_회의_assemblyai1 (...   
1  /content/work/outputs/1106_오후_회의_senko (recomm...   

                                speaker_profiles_csv  \
0  /content/work/outputs/1106_오후_회의_assemblyai1 (...   
1  /content/work/outputs/1106_오후_회의_senko (recomm...   

                               speaker_profiles_json  \
0  /content/work/outputs/1106_오후_회의_assemblyai1 (...   
1  /content/work/outputs/1106_오후_회의_senko (recomm...   

                                representatives_json  
0  /content/work/outputs/1106_오후_회의_assemblyai1 (...  
1  /content/work/outputs/1106_오후_회의_senko (recomm...  
